DPO 用二元分类的损失直接把“更好的回复”相对“更差的回复”的对数几率推高，同时用一个参考模型（通常是 SFT 后模型）做软约束，从而在不做强化学习、不训练显式奖励模型的前提下达成 RLHF 同样的目标。它等价于在 KL 正则下的“最大化偏好概率”的闭式解，用起来和普通监督学习一样稳定高效。

**RLHF 的标准目标**

在偏好学习里，经常先拟合一个奖励模型 $r_\phi(x,y)$，再用 RL（例如 PPO）优化策略 $\pi_\theta$
$$\max_{\pi_\theta}\ \mathbb{E}{x\sim\mathcal D,\ y\sim \pi\theta(\cdot|x)}[r_\phi(x,y)]\ -\ \beta\, D_{\mathrm{KL}}\!\left(\pi_\theta(\cdot|x)\ \|\ \pi_{\text{ref}}(\cdot|x)\right)$$
其中 $\beta$ 控制“别离参考分布太远”的强度（防止模式崩塌、奖励黑客）。这正是 InstructGPT 等 RLHF 做法的核心形式。 

**Bradley–Terry 偏好模型**

偏好数据通常是成对比较对同一提示 $x$，人类在两个回答 $y_w$（赢）与 $y_\ell$（输）之间做选择。Bradley–Terry（BT）模型假设偏好分布满足
$$p(y_w \succ y_\ell\mid x)=\sigma\!\big(r(x,y_w)-r^*(x,y_\ell)\big)$$
$\sigma$ 为 Sigmoid。BT（或更一般的 Plackett–Luce 排序模型）是偏好建模的经典选择。 ￼

**换元**

对 KL 正则化的 RL 目标，最优策略具有能量型形式（Boltzmann policy）
$$\pi_r(y|x)=\frac{1}{Z(x)}\,\pi_{\text{ref}}(y|x)\exp\!\Big(\frac{1}{\beta}r(x,y)\Big)$$

改写可得等价的奖励参数化
$$r(x,y)=\beta\log\frac{\pi_r(y|x)}{\pi_{\text{ref}}(y|x)}+\beta\log Z(x)$$
把它代回 BT 模型，配对差分里那项 $\log Z(x)$ 会抵消，于是偏好概率可以完全用“策略—参考策略的对数几率差”表示。

于是我们可以直接用策略参数 \pi_\theta 拟合偏好最大似然（相当于二元交叉熵）
$$\mathcal L_{\text{DPO}}(\theta)\ =\ -\ \mathbb{E}{(x,y_w,y\ell)\sim\mathcal D}
\left[\ \log \sigma\!\Big(\underbrace{\beta\!\left[\log\frac{\pi_\theta(y_w|x)}{\pi_{\text{ref}}(y_w|x)}-\log\frac{\pi_\theta(y_\ell|x)}{\pi_{\text{ref}}(y_\ell|x)}\right]}_{\text{隐式“优势”差}}\Big)\ \right]$$
直观上，它提高赢样本的相对对数几率、降低输样本的相对对数几率；而 $\beta$ 决定“贴近参考”的强度。其梯度会对“当前排序错得越厉害”的样本给更大权重，从而稳定训练。 ￼
  ￼

- 数据：尽量保证偏好标注一致、覆盖面广；公开集如 Anthropic HH 可直接用。 ￼
- 参考模型：优先用同一基座的 SFT 权重；没有就先对“赢样本”做一轮 MLE 近似。 ￼
- 目标计算：按整段序列总 logprob计算对数几率差更契合论文设定；是否做长度归一化可按经验验证（SimPO 倾向平均；RainbowPO 给出对比）。 ￼ ￼
- 超参：$\beta$ 先从 0.1–0.5 网格搜索；batch 尽量大一些以稳定梯度。 ￼
- 评测：既看“偏好胜率 / RewardBench 这类偏好指标”，也要看多样性、长度、事实性 / 安全性，避免只优化“赢率”导致模式收缩。

In [2]:
import torch
import torch.nn.functional as F

beta = 0.4
chosen_log_probs = F.log_softmax(torch.randn(2, 8), dim=1)
chosen_old_log_probs = F.log_softmax(torch.randn(2, 8), dim=1)
rejected_log_probs = F.log_softmax(torch.randn(2, 8), dim=1)
rejected_old_log_probs = F.log_softmax(torch.randn(2, 8), dim=1)


In [4]:
logits = (chosen_log_probs - chosen_old_log_probs) - (rejected_log_probs - rejected_old_log_probs)
losses = -F.logsigmoid(beta * logits)
losses.mean()

tensor(0.8138)

**为什么 $\beta$ 越大，惩罚越重**

约束 / 拉格朗日视角。

$$\max_{\pi}\ \mathbb{E}[r(x,y)]\ -\ \beta\,D_{\mathrm{KL}}\!\big(\pi(\cdot|x)\ \|\ \pi_{\text{ref}}(\cdot|x)\big)$$
β 就是 KL 惩罚的权重。β 越大，对“远离参考”的惩罚越重，最优解自然更贴着 $\pi_{\text{ref}}$。

Boltzmann 能量视角。该目标的最优策略满足
$$\pi^*(y|x)\ \propto\ \pi_{\text{ref}}(y|x)\,\exp\!\big(r(x,y)/\beta\big)$$
把 $\beta$ 看作“温度”则有
- $\beta\to\infty: \exp(r/\beta)\approx 1+r/\beta$，正规化后 $\pi^*\to \pi_{\text{ref}}$。
- $\beta\to 0^+: \exp(r/\beta)$ 变得极端尖锐，$\pi^*$ 几乎只压在高奖励的 $y$ 上，强烈偏离参考分布。

DPO 的“换元”视角。DPO 用
$$r(x,y)=\beta\Big[\log \pi_\theta(y|x)-\log \pi_{\text{ref}}(y|x)\Big]+\beta\log Z(x)$$
把奖励写成相对参考的对数几率。对一对偏好 $(y_w,y_\ell)$，有
$$r(x,y_w)-r(x,y_\ell)=\beta\Big[\log\tfrac{\pi_\theta}{\pi_{\text{ref}}}(y_w|x)-\log\tfrac{\pi_\theta}{\pi_{\text{ref}}}(y_\ell|x)\Big]$$
要匹配同样的“偏好强度”（固定的 r 差），所需的“相对对数几率差” $\Delta\log\text{ratio}=\frac{r(x,y_w)-r(x,y_\ell)}{\beta}$ 会随着 β 增大而按 $1/\beta$ 缩小。也就是说，在 β 更大的设定下，实现同样的偏好拟合所需的“从参考分布出发的改动”更小，策略就更贴近 $\pi_{\text{ref}}$。